In [1]:
import os 

In [2]:
%pwd

'c:\\Users\\amanbatra\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\amanbatra\\Chicken-Disease-Classification'

In [5]:
import tensorflow as tf

In [6]:
model = tf.keras.models.load_model("artifacts/training/model.keras")

In [55]:

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    params_image_size: list
    params_batch_size: int

In [56]:
from Chicken_Disease_Classifier.constants import *
from Chicken_Disease_Classifier.utils.common import read_yaml, create_directories, save_json

In [57]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.keras",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [58]:
from pathlib import Path
from Chicken_Disease_Classifier.entity.config_entity import EvaluationConfig

def get_validation_config(self) -> EvaluationConfig:
    cfg = self.config
    params = self.params

    model_path = Path(cfg.training.trained_model_path)  # artifacts/training/model.keras
    # fallback to checkpoint if final model missing
    if not model_path.exists():
        model_path = Path(cfg.prepare_callbacks.checkpoint_model_filepath)  # artifacts/prepare_callbacks/checkpoints/model.keras

    return EvaluationConfig(
        path_of_model=model_path,
        training_data=Path(cfg.data_ingestion.unzip_dir),
        params_image_size=tuple(params.IMAGE_SIZE),
        params_batch_size=int(params.BATCH_SIZE),
    )



In [59]:
# add near your other @dataclass configs
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass
class EvaluationConfig:
    path_of_model: Path          # e.g., artifacts/training/model.keras  (or checkpoint)
    training_data: Path          # e.g., artifacts/data_ingestion
    params_image_size: List[int] # e.g., [224, 224, 3]
    params_batch_size: int       # e.g., 32


In [60]:

from urllib.parse import urlparse

In [61]:
from pathlib import Path
import tensorflow as tf
from Chicken_Disease_Classifier.utils.common import save_json

class Evaluation:
    def __init__(self, config):
        self.config = config
        self.model = None
        self.valid_generator = None
        self.score = None

    def _valid_generator(self):
        datagenerator_kwargs = dict(rescale=1./255, validation_split=0.30)
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear",
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(**datagenerator_kwargs)
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        p = Path(path)
        if not p.exists():
            raise FileNotFoundError(f"Model file not found: {p.resolve()}")
        if p.suffix.lower() == ".h5":
            raise ValueError(f"Found HDF5 file ({p}). Use a V3 '.keras' file with Keras 3.")
        return tf.keras.models.load_model(str(p))  # pass string

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)  # <-- FIXED

    def save_score(self):
        if self.score is None:
            raise RuntimeError("Call evaluation() before save_score().")
        scores = {"loss": float(self.score[0]), "accuracy": float(self.score[1])}
        save_json(path=Path("scores.json"), data=scores)


In [62]:

try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()

except Exception as e:
   raise e

[2025-10-24 03:13:25,783: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-10-24 03:13:25,787: INFO: common: yaml file: params.yaml loaded successfully]
[2025-10-24 03:13:25,789: INFO: common: created directory at: artifacts]
Found 116 images belonging to 2 classes.


c:\Users\amanbatra\Chicken-Disease-Classification\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


8/8 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.5086 - loss: 0.9356
[2025-10-24 03:13:39,001: INFO: common: json file saved at: scores.json]
